For this project, we use the dataset about egyptians hepatitis C Virus (HCV). The dataset is available here http://archive.ics.uci.edu/ml/datasets/Hepatitis+C+Virus+%28HCV%29+for+Egyptian+patients.
Let us load useful librairies.
The dataset is described in this paper https://www.researchgate.net/profile/Mahmoud_Nasr15/publication/323130913_A_novel_model_based_on_non_invasive_methods_for_prediction_of_liver_fibrosis/links/5e27008ca6fdcc70a13dbfcb/A-novel-model-based-on-non-invasive-methods-for-prediction-of-liver-fibrosis.pdf

In [2]:
from pyspark.sql import SQLContext
from pyspark import SparkContext
from pyspark.sql.functions import col
from pyspark.sql import functions as fn
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
from pyspark.ml.linalg import Vectors 
from pyspark.ml import Pipeline
from pyspark.ml.feature import IndexToString,StringIndexer, VectorIndexer
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
import seaborn as sn
from pyspark.ml.feature import StandardScaler
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


import functools
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, VectorIndexer, SQLTransformer, IndexToString
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression  as LR
from pyspark.ml.classification import RandomForestClassifier as RF 
from pyspark.ml.classification import NaiveBayes as NB

Let us read the dataset.

In [4]:
dataset = spark.read.load("/FileStore/tables/HCV_Egy_Data-4d319.csv", format = "csv", header = True, inferSchema = True)
display(dataset)


Age,Gender,BMI,Fever,Nausea/Vomting,Headache,Diarrhea,Fatigue & generalized bone ache,Jaundice,Epigastric pain,WBC,RBC,HGB,Plat,AST 1,ALT 1,ALT4,ALT 12,ALT 24,ALT 36,ALT 48,ALT after 24 w,RNA Base,RNA 4,RNA 12,RNA EOT,RNA EF,Baseline histological Grading,Baselinehistological staging
56,1,35,2,1,1,1,2,2,2,7425,4248807.0,14,112132.0,99,84,52.0,109,81,5,5,5,655330,634536,288194,5,5,13,2
46,1,29,1,2,2,1,2,2,1,12101,4429425.0,10,129367.0,91,123,95.0,75,113,57,123,44,40620,538635,637056,336804,31085,4,2
57,1,33,2,2,2,2,1,1,1,4178,4621191.0,12,151522.0,113,49,95.0,107,116,5,5,5,571148,661346,5,735945,558829,4,4
49,2,33,1,2,1,2,1,2,1,6490,4794631.0,10,146457.0,43,64,109.0,80,88,48,77,33,1041941,449939,585688,744463,582301,10,3
59,1,32,1,1,2,1,2,2,2,3661,4606375.0,11,187684.0,99,104,67.0,48,120,94,90,30,660410,738756,3731527,338946,242861,11,1
58,2,22,2,2,2,1,2,2,1,11785,3882456.0,15,131228.0,66,104,121.0,96,65,73,114,29,1157452,1086852,5,5,5,4,4
42,2,26,1,1,2,2,2,2,2,11620,4747333.0,12,177261.0,78,57,113.0,118,107,84,80,28,325694,1034008,275095,214566,635157,12,4
48,2,30,1,1,2,2,1,1,2,7335,4405941.0,11,216176.0,119,112,80.0,127,45,96,53,39,641129,72050,787295,370605,506296,12,3
44,1,23,1,1,2,2,2,1,2,10480,4608464.0,12,148889.0,93,83,55.0,102,97,122,39,45,591441,757361,5,371090,203042,5,2
45,1,30,2,1,2,2,1,1,2,6681,4455329.0,12,98200.0,55,68,72.0,127,81,125,43,30,1151206,230488,267320,275295,555516,4,2


We want to see the schema of the dataframe

In [6]:
dataset.printSchema()

root
-- Age : integer (nullable = true)
-- Gender: integer (nullable = true)
-- BMI: integer (nullable = true)
-- Fever: integer (nullable = true)
-- Nausea/Vomting: integer (nullable = true)
-- Headache : integer (nullable = true)
-- Diarrhea : integer (nullable = true)
-- Fatigue & generalized bone ache : integer (nullable = true)
-- Jaundice : integer (nullable = true)
-- Epigastric pain : integer (nullable = true)
-- WBC: integer (nullable = true)
-- RBC: double (nullable = true)
-- HGB: integer (nullable = true)
-- Plat: double (nullable = true)
-- AST 1: integer (nullable = true)
-- ALT 1: integer (nullable = true)
-- ALT4: double (nullable = true)
-- ALT 12: integer (nullable = true)
-- ALT 24: integer (nullable = true)
-- ALT 36: integer (nullable = true)
-- ALT 48: integer (nullable = true)
-- ALT after 24 w: integer (nullable = true)
-- RNA Base: integer (nullable = true)
-- RNA 4: integer (nullable = true)
-- RNA 12: integer (nullable = true)
-- RNA EOT: integer (nullable = true)
-- RNA EF: integer (nullable = true)
-- Baseline histological Grading: integer (nullable = true)
-- Baselinehistological staging: integer (nullable = true)

How many columns we have?

In [8]:
len(dataset.columns)

Out[5]: 29

Since label column is named Baseline histological Staging, we rename this variable.

In [10]:
dataset.columns

Out[6]: ['Age ',
 'Gender',
 'BMI',
 'Fever',
 'Nausea/Vomting',
 'Headache ',
 'Diarrhea ',
 'Fatigue & generalized bone ache ',
 'Jaundice ',
 'Epigastric pain ',
 'WBC',
 'RBC',
 'HGB',
 'Plat',
 'AST 1',
 'ALT 1',
 'ALT4',
 'ALT 12',
 'ALT 24',
 'ALT 36',
 'ALT 48',
 'ALT after 24 w',
 'RNA Base',
 'RNA 4',
 'RNA 12',
 'RNA EOT',
 'RNA EF',
 'Baseline histological Grading',
 'Baselinehistological staging']

Now, we want to use some aggregation functions in order to explore the dataset. Before, transform label 'Baselinehistological staging' into string type.

In [12]:
def string_to_float(x):
    return float(x)
  
def condition(r):
    if (r==1):
        return "F1"
    elif(r== 2):
        return "F2"
    elif (r==3):
        return "F3"
    else:
        return "F4"


from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, DoubleType
string_to_float_udf = udf(string_to_float, DoubleType())
quality_udf = udf(lambda x: condition(x), StringType())
Label = quality_udf("Baselinehistological staging")
dataset = dataset.withColumn("Label", fn.lit(Label)) ## Be careful here, please remove label column
display(dataset)

Age,Gender,BMI,Fever,Nausea/Vomting,Headache,Diarrhea,Fatigue & generalized bone ache,Jaundice,Epigastric pain,WBC,RBC,HGB,Plat,AST 1,ALT 1,ALT4,ALT 12,ALT 24,ALT 36,ALT 48,ALT after 24 w,RNA Base,RNA 4,RNA 12,RNA EOT,RNA EF,Baseline histological Grading,Baselinehistological staging,Label
56,1,35,2,1,1,1,2,2,2,7425,4248807.0,14,112132.0,99,84,52.0,109,81,5,5,5,655330,634536,288194,5,5,13,2,F2
46,1,29,1,2,2,1,2,2,1,12101,4429425.0,10,129367.0,91,123,95.0,75,113,57,123,44,40620,538635,637056,336804,31085,4,2,F2
57,1,33,2,2,2,2,1,1,1,4178,4621191.0,12,151522.0,113,49,95.0,107,116,5,5,5,571148,661346,5,735945,558829,4,4,F4
49,2,33,1,2,1,2,1,2,1,6490,4794631.0,10,146457.0,43,64,109.0,80,88,48,77,33,1041941,449939,585688,744463,582301,10,3,F3
59,1,32,1,1,2,1,2,2,2,3661,4606375.0,11,187684.0,99,104,67.0,48,120,94,90,30,660410,738756,3731527,338946,242861,11,1,F1
58,2,22,2,2,2,1,2,2,1,11785,3882456.0,15,131228.0,66,104,121.0,96,65,73,114,29,1157452,1086852,5,5,5,4,4,F4
42,2,26,1,1,2,2,2,2,2,11620,4747333.0,12,177261.0,78,57,113.0,118,107,84,80,28,325694,1034008,275095,214566,635157,12,4,F4
48,2,30,1,1,2,2,1,1,2,7335,4405941.0,11,216176.0,119,112,80.0,127,45,96,53,39,641129,72050,787295,370605,506296,12,3,F3
44,1,23,1,1,2,2,2,1,2,10480,4608464.0,12,148889.0,93,83,55.0,102,97,122,39,45,591441,757361,5,371090,203042,5,2,F2
45,1,30,2,1,2,2,1,1,2,6681,4455329.0,12,98200.0,55,68,72.0,127,81,125,43,30,1151206,230488,267320,275295,555516,4,2,F2


In [13]:
dataset.printSchema()

root
-- Age : integer (nullable = true)
-- Gender: integer (nullable = true)
-- BMI: integer (nullable = true)
-- Fever: integer (nullable = true)
-- Nausea/Vomting: integer (nullable = true)
-- Headache : integer (nullable = true)
-- Diarrhea : integer (nullable = true)
-- Fatigue & generalized bone ache : integer (nullable = true)
-- Jaundice : integer (nullable = true)
-- Epigastric pain : integer (nullable = true)
-- WBC: integer (nullable = true)
-- RBC: double (nullable = true)
-- HGB: integer (nullable = true)
-- Plat: double (nullable = true)
-- AST 1: integer (nullable = true)
-- ALT 1: integer (nullable = true)
-- ALT4: double (nullable = true)
-- ALT 12: integer (nullable = true)
-- ALT 24: integer (nullable = true)
-- ALT 36: integer (nullable = true)
-- ALT 48: integer (nullable = true)
-- ALT after 24 w: integer (nullable = true)
-- RNA Base: integer (nullable = true)
-- RNA 4: integer (nullable = true)
-- RNA 12: integer (nullable = true)
-- RNA EOT: integer (nullable = true)
-- RNA EF: integer (nullable = true)
-- Baseline histological Grading: integer (nullable = true)
-- Baselinehistological staging: integer (nullable = true)
-- Label: string (nullable = true)

In [14]:
label = dataset.select("Label")
label.show(5)

+-----+
Label|
+-----+
 F2|
 F2|
 F4|
 F3|
 F1|
+-----+
only showing top 5 rows

In the dataset, there are qualitatives variables and quantitatives variables. We separate qualitative and quantitative variables. In the qualitative variables, 1 means presence of disease whereas 2 is absence of disease.
For Gender variable, 1 is male and 2 is female.

In [16]:
quali_dataset =  dataset.select('Gender', 'Fever', 'Nausea/Vomting','Headache ','Diarrhea ','Fatigue & generalized bone ache ', 'Jaundice ','Epigastric pain ') 
quanti_dataset = dataset.drop('Gender', 'Fever', 'Nausea/Vomting','Headache ','Diarrhea ','Fatigue & generalized bone ache ', 'Jaundice ','Epigastric pain ', 'label_new')
display(quali_dataset)
dispaly(quanti_dataset)

Gender,Fever,Nausea/Vomting,Headache,Diarrhea,Fatigue & generalized bone ache,Jaundice,Epigastric pain
1,2,1,1,1,2,2,2
1,1,2,2,1,2,2,1
1,2,2,2,2,1,1,1
2,1,2,1,2,1,2,1
1,1,1,2,1,2,2,2
2,2,2,2,1,2,2,1
2,1,1,2,2,2,2,2
2,1,1,2,2,1,1,2
1,1,1,2,2,2,1,2
1,2,1,2,2,1,1,2


In [17]:
for i in quanti_dataset.columns:
  quanti_dataset.describe(i).show()

+-------+-----------------+
summary| Age |
+-------+-----------------+
 count| 1385|
 mean|46.31913357400722|
 stddev|8.781505746704225|
 min| 32|
 max| 61|
+-------+-----------------+

+-------+-----------------+
summary| BMI|
+-------+-----------------+
 count| 1385|
 mean| 28.6086642599278|
 stddev|4.076215378604909|
 min| 22|
 max| 35|
+-------+-----------------+

+-------+------------------+
summary| WBC|
+-------+------------------+
 count| 1385|
 mean| 7533.386281588448|
 stddev|2668.2203328182695|
 min| 2991|
 max| 12101|
+-------+------------------+

+-------+------------------+
summary| RBC|
+-------+------------------+
 count| 1385|
 mean| 4422129.610830325|
 stddev|346357.71159949043|
 min| 3816422.0|
 max| 5018451.0|
+-------+------------------+

+-------+------------------+
summary| HGB|
+-------+------------------+
 count| 1385|
 mean|12.587725631768953|
 stddev| 1.713511280186577|
 min| 10|
 max| 15|
+-------+------------------+

+-------+-----------------+
summary| Plat|
+-------+-----------------+
 count| 1385|
 mean|158348.0606498195|
 stddev| 38794.7855502621|
 min| 93013.0|
 max| 226464.0|
+-------+-----------------+

+-------+------------------+
summary| AST 1|
+-------+------------------+
 count| 1385|
 mean| 82.77472924187725|
 stddev|25.993242167609793|
 min| 39|
 max| 128|
+-------+------------------+

+-------+------------------+
summary| ALT 1|
+-------+------------------+
 count| 1385|
 mean| 83.91624548736462|
 stddev|25.922799521394236|
 min| 39|
 max| 128|
+-------+------------------+

+-------+------------------+
summary| ALT4|
+-------+------------------+
 count| 1385|
 mean| 83.4057761732852|
 stddev|26.529729667056376|
 min| 39.0|
 max| 128.0|
+-------+------------------+

+-------+-----------------+
summary| ALT 12|
+-------+-----------------+
 count| 1385|
 mean|83.51046931407942|
 stddev|26.06447788401469|
 min| 39|
 max| 128|
+-------+-----------------+

+-------+------------------+
summary| ALT 24|
+-------+------------------+
 count| 1385|
 mean| 83.70902527075812|
 stddev|26.205994454785436|
 min| 39|
 max| 128|
+-------+------------------+

+-------+-----------------+
summary| ALT 36|
+-------+-----------------+
 count| 1385|
 mean|83.11768953068592|
 stddev|26.39903079712999|
 min| 5|
 max| 128|
+-------+-----------------+

+-------+------------------+
summary| ALT 48|
+-------+------------------+
 count| 1385|
 mean| 83.62960288808664|
 stddev|26.223954685295038|
 min| 5|
 max| 128|
+-------+------------------+

+-------+-----------------+
summary| ALT after 24 w|
+-------+-----------------+
 count| 1385|
 mean|33.43826714801444|
 stddev|7.073569415669241|
 min| 5|
 max| 45|
+-------+-----------------+

+-------+-----------------+
summary| RNA Base|
+-------+-----------------+
 count| 1385|
 mean|590951.2187725632|
 stddev|353935.3576019475|
 min| 11|
 max| 1201086|
+-------+-----------------+

+-------+------------------+
summary| RNA 4|
+-------+------------------+
 count| 1385|
 mean| 600895.6498194946|
 stddev|362315.13278605294|
 min| 5|
 max| 1201715|
+-------+------------------+

+-------+------------------+
summary| RNA 12|
+-------+------------------+
 count| 1385|
 mean|288753.61227436824|
 stddev|285350.67451137374|
 min| 5|
 max| 3731527|
+-------+------------------+

+-------+------------------+
summary| RNA EOT|
+-------+------------------+
 count| 1385|
 mean|287660.33646209387|
 stddev| 264559.5250695325|
 min| 5|
 max| 808450|
+-------+------------------+

+-------+------------------+
summary| RNA EF|
+-------+------------------+
 count| 1385|
 mean| 291378.2909747292|
 stddev|267700.69171339076|
 min| 5|
 max| 810333|
+-------+------------------+

+-------+-----------------------------+
summary|Baseline histological Grading|
+-------+-----------------------------+
 count| 1385|
 mean| 9.76173285198556|
 stddev| 4.023896452419502|
 min| 3|
 max| 16|
+-------+-----------------------------+

+-------+----------------------------+
summary|Baselinehistological staging|

In [18]:
for i in quali_dataset.columns:
  quali_dataset.select(i).distinct().show()

+------+
Gender|
+------+
 1|
 2|
+------+

+-----+
Fever|
+-----+
 1|
 2|
+-----+

+--------------+
Nausea/Vomting|
+--------------+
 1|
 2|
+--------------+

+---------+
Headache |
+---------+
 1|
 2|
+---------+

+---------+
Diarrhea |
+---------+
 1|
 2|
+---------+

+--------------------------------+
Fatigue & generalized bone ache |
+--------------------------------+
 1|
 2|
+--------------------------------+

+---------+
Jaundice |
+---------+
 1|
 2|
+---------+

+----------------+
Epigastric pain |
+----------------+
 1|
 2|
+----------------+

In [19]:
label.distinct().show()

+-----+
Label|
+-----+
 F2|
 F1|
 F4|
 F3|
+-----+

We have 4 differents values of label.

In [21]:
for i in quali_dataset.columns:
  quali_dataset.groupby('Gender').agg(fn.count(i)).show()

+------+-------------+
Gender|count(Gender)|
+------+-------------+
 1| 707|
 2| 678|
+------+-------------+

+------+------------+
Gender|count(Fever)|
+------+------------+
 1| 707|
 2| 678|
+------+------------+

+------+---------------------+
Gender|count(Nausea/Vomting)|
+------+---------------------+
 1| 707|
 2| 678|
+------+---------------------+

+------+----------------+
Gender|count(Headache )|
+------+----------------+
 1| 707|
 2| 678|
+------+----------------+

+------+----------------+
Gender|count(Diarrhea )|
+------+----------------+
 1| 707|
 2| 678|
+------+----------------+

+------+---------------------------------------+
Gender|count(Fatigue & generalized bone ache )|
+------+---------------------------------------+
 1| 707|
 2| 678|
+------+---------------------------------------+

+------+----------------+
Gender|count(Jaundice )|
+------+----------------+
 1| 707|
 2| 678|
+------+----------------+

+------+-----------------------+
Gender|count(Epigastric pain )|
+------+-----------------------+
 1| 707|
 2| 678|
+------+-----------------------+

Let's inspect the NaN in the the quantitative and qualitative variable.

In [23]:
for col in quali_dataset.columns:
  print(col, "\t", "with null values: ", quali_dataset.filter(quali_dataset[col].isNull()).count())

Gender with null values: 0
Fever with null values: 0
Nausea/Vomting with null values: 0
Headache with null values: 0
Diarrhea with null values: 0
Fatigue & generalized bone ache with null values: 0
Jaundice with null values: 0
Epigastric pain with null values: 0

In [24]:
for col in quanti_dataset.columns:
  print(col, "\t", "with null values: ", quanti_dataset.filter(quanti_dataset[col].isNull()).count())

Age with null values: 0
BMI with null values: 0
WBC with null values: 0
RBC with null values: 0
HGB with null values: 0
Plat with null values: 0
AST 1 with null values: 0
ALT 1 with null values: 0
ALT4 with null values: 0
ALT 12 with null values: 0
ALT 24 with null values: 0
ALT 36 with null values: 0
ALT 48 with null values: 0
ALT after 24 w with null values: 0
RNA Base with null values: 0
RNA 4 with null values: 0
RNA 12 with null values: 0
RNA EOT with null values: 0
RNA EF with null values: 0
Baseline histological Grading with null values: 0
Baselinehistological staging with null values: 0
Label with null values: 0

In [25]:
for i in quali_dataset.columns[1:]:
  print( "\t", "The number of persons which have ", i,":", quali_dataset.filter(quali_dataset[i]==1).count())

The number of persons which have Fever : 671
 The number of persons which have Nausea/Vomting : 689
 The number of persons which have Headache : 698
 The number of persons which have Diarrhea : 689
 The number of persons which have Fatigue & generalized bone ache : 694
 The number of persons which have Jaundice : 691
 The number of persons which have Epigastric pain : 687

In [26]:
Occurence_by_class = dataset.groupby('Label').count().collect()
categories = [i[0] for i in Occurence_by_class]
counts = [i[1] for i in Occurence_by_class]
ind = np.array(range(len(categories)))
width = 0.35
plt.bar(ind, counts, width = width, color = 'r')
plt.ylabel('counts')
plt.title('Label distribution')
plt.xticks(ind, categories)
display()


What is distribution of the observation by gender?

In [28]:
Occurence_by_Gender = dataset.groupby('Gender').count().collect()
categories_Gender = [i[0] for i in Occurence_by_Gender]
counts_Gender = [i[1] for i in Occurence_by_Gender]
ind_Gender = np.array(range(len(categories_Gender)))
width = 0.35
plt.bar(ind_Gender, counts_Gender, width = width, color = 'r')
plt.ylabel('counts')
plt.title('Gender distribution')
plt.xticks(ind_Gender, categories_Gender)
display()

There is more male than female. 
Let's plot some numeric variables.

In [30]:
data = dataset.select('Plat','AST 1','ALT 1','ALT4', 'ALT 12','ALT 24','ALT 36','ALT 48','ALT after 24 w','RNA Base','RNA 4','RNA 12','RNA EOT','RNA EF','Baseline histological Grading')
data_toPandas = data.toPandas()
for i in data.columns:
  sn.boxplot(data_toPandas[i])
  display()

In [31]:
for i in data.columns:
  sn.distplot(data_toPandas[i])
  display()


According to these plot above, one can say that ALT after 24w and RNA 12 have outliers. The distribution of variable Plat seems centered.
Now, we want to deal with categorical features. We will use one hot encoders to convert catogorical variables.

In [33]:
def transData(data):
  return data.rdd.map(lambda r: [Vectors.dense(r[:-1]),r[-1]]).toDF(['features','Label'])
dataset.columns

Out[27]: ['Age ',
 'Gender',
 'BMI',
 'Fever',
 'Nausea/Vomting',
 'Headache ',
 'Diarrhea ',
 'Fatigue & generalized bone ache ',
 'Jaundice ',
 'Epigastric pain ',
 'WBC',
 'RBC',
 'HGB',
 'Plat',
 'AST 1',
 'ALT 1',
 'ALT4',
 'ALT 12',
 'ALT 24',
 'ALT 36',
 'ALT 48',
 'ALT after 24 w',
 'RNA Base',
 'RNA 4',
 'RNA 12',
 'RNA EOT',
 'RNA EF',
 'Baseline histological Grading',
 'Baselinehistological staging',
 'Label']

In [34]:
transformed = transData(dataset)
transformed.show(5, True)

+--------------------+-----+
 features|Label|
+--------------------+-----+
[56.0,1.0,35.0,2....| F2|
[46.0,1.0,29.0,1....| F2|
[57.0,1.0,33.0,2....| F4|
[49.0,2.0,33.0,1....| F3|
[59.0,1.0,32.0,1....| F1|
+--------------------+-----+
only showing top 5 rows

Standardization

In [36]:
scaler = StandardScaler(inputCol="features", outputCol="Features",
                        withStd=True, withMean=False)
scalerModel = scaler.fit(transformed)
scaledData = scalerModel.transform(transformed)
transformed = scaledData.select("Features", "Label")
transformed.show(5, True)

+--------------------+-----+
 Features|Label|
+--------------------+-----+
[6.37703847327291...| F2|
[5.23828160304560...| F2|
[6.49091416029564...| F4|
[5.57990866411379...| F3|
[6.71866553434110...| F1|
+--------------------+-----+
only showing top 5 rows

In [37]:
labelIndexer = StringIndexer(inputCol='Label',
                             outputCol='indexedLabel').fit(transformed)
labelIndexer.transform(transformed).show(5, True)

+--------------------+-----+------------+
 Features|Label|indexedLabel|
+--------------------+-----+------------+
[6.37703847327291...| F2| 3.0|
[5.23828160304560...| F2| 3.0|
[6.49091416029564...| F4| 0.0|
[5.57990866411379...| F3| 1.0|
[6.71866553434110...| F1| 2.0|
+--------------------+-----+------------+
only showing top 5 rows

In [38]:
featureIndexer =VectorIndexer(inputCol="Features", \
                              outputCol="indexedFeatures", \
                              maxCategories=4).fit(transformed)
featureIndexer.transform(transformed).show(5, True)

+--------------------+-----+--------------------+
 Features|Label| indexedFeatures|
+--------------------+-----+--------------------+
[6.37703847327291...| F2|[6.37703847327291...|
[5.23828160304560...| F2|[5.23828160304560...|
[6.49091416029564...| F4|[6.49091416029564...|
[5.57990866411379...| F3|[5.57990866411379...|
[6.71866553434110...| F1|[6.71866553434110...|
+--------------------+-----+--------------------+
only showing top 5 rows

Let's split the data to training and test data sets.

In [40]:
train_data, test_data = transformed.randomSplit([0.6,0.4], seed = 0.0)

In [41]:
train_data.show(5, False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
Features |Label|
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
[3.6440219847273774,1.9997162599711977,5.887814497234451,2.0002421090209084,3.9986067673556756,1.9993409073190969,3.9986067673556738,3.9985650762467175,1.9992825381233597,3.998681814638193,4.028153098079265,11.885371285626825,5.835969751486632,2.9274295086091198,3.577853020424185,2.121684425118058,4.485533832927432,3.2227770060768073,4.12119449183041,3.6364971402827706,3.4319766442575155,4.099768913804639,1.6599754372682862,0.3226887022382206,1.752229956547976E-5,1.889933843314045E-5,1.8677575944978003E-5,2.485153412431169,1.7834971611290327] |F2 |
[3.6440219847273774,1.9997162599711977,5.887814497234451,4.000484218041817,3.9986067673556756,1.9993409073190969,3.9986067673556738,1.9992825381233588,3.9985650762467193,1.9993409073190964,1.2322820419133442,13.867948768394239,6.419566726635296,4.174813643193492,4.2318691639425845,4.70628181571642,3.317033422669026,2.455449147487091,4.350149741376544,3.522856604648934,1.7159883221287577,3.534283546383309,2.399511610691158,2.0211272832272624,2.4212488762575584,1.6893589443908121,2.6712893247418883,0.7455460237293507,3.5669943222580653] |F4 |
[3.6440219847273774,1.9997162599711977,6.378465705337321,2.0002421090209084,3.9986067673556756,1.9993409073190969,1.9993033836778369,1.9992825381233588,1.9992825381233597,1.9993409073190964,3.0162426622015186,12.904519952390677,7.586760676932623,3.61107808724705,2.9623084147598093,3.780455884755813,3.8447432853663708,3.9901048646665234,3.2816919101612525,2.1970503555875074,4.194638120759185,3.534283546383309,1.618967949069486,3.160508343095295,1.752229956547976E-5,1.889933843314045E-5,1.8677575944978003E-5,2.7336687536742863,0.8917485805645163]|F1 |
[3.6440219847273774,1.9997162599711977,6.378465705337321,2.0002421090209084,3.9986067673556756,3.9986818146381937,3.9986067673556738,3.9985650762467175,1.9992825381233597,3.998681814638193,3.324313172679855,13.912125639552498,8.170357652081286,3.4976092295755254,3.8856253232563733,2.430293068771594,3.317033422669026,2.8007466838524633,4.579104990922678,1.4773269632398756,3.8895735301585175,3.3929122045279767,3.148956373139329,1.4306440805112108,2.143176991073221,1.7239674129295788,1.5044899489135453,3.727730118646754,0.8917485805645163] |F1 |
[3.6440219847273774,1.9997162599711977,7.114442517491628,2.0002421090209084,3.9986067673556756,3.9986818146381937,1.9993033836778369,3.9985650762467175,3.9985650762467193,3.998681814638193,2.4349563340361913,14.100295262509713,7.00316370178396,5.356544624554472,2.3467638090954335,2.198836586031442,3.693969038881415,2.0717852181922334,3.2816919101612525,3.901658390095056,3.927706603983601,3.6756548882386415,0.7000120069344458,2.53997947290659,1.752229956547976E-5,1.889933843314045E-5,1.8677575944978003E-5,1.4910920474587015,3.5669943222580653] |F4 |
+------------------------------------------------------------------------------------------------------

In [42]:
test_data.show(5, False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
Features |Label|
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
[3.6440219847273774,1.9997162599711977,5.39716328913158,2.0002421090209084,3.9986067673556756,3.9986818146381937,3.9986067673556738,1.9992825381233588,1.9992825381233597,1.9993409073190964,1.1816865201194569,11.990092499520111,6.419566726635296,5.3887138963341314,2.654536111927621,3.548999402015661,1.583129588092035,3.8750056858780657,4.083035283572721,2.954653926479751,4.270904268409352,4.241140255659971,1.4743963517397072,0.2651365932781096,1.752229956547976E-5,1.889933843314045E-5,1.8677575944978003E-5,1.4910920474587015,0.8917485805645163] |F1 |
[3.6440219847273774,1.9997162599711977,6.133140101285886,2.0002421090209084,1.9993033836778378,1.9993409073190969,1.9993033836778369,3.9985650762467175,1.9992825381233597,3.998681814638193,1.345091316431567,11.335861360985435,7.586760676932623,3.3283854561512736,2.654536111927621,3.819031965212505,2.9777913680778756,3.4146089707242364,3.319851118418941,1.742488213052161,1.8685206174290918,6.220339041634625,1.8210359212680525,3.01118805502456,1.752229956547976E-5,1.889933843314045E-5,1.8677575944978003E-5,0.7455460237293507,1.7834971611290327] |F2 |
[3.6440219847273774,1.9997162599711977,6.133140101285886,2.0002421090209084,1.9993033836778378,1.9993409073190969,3.9986067673556738,3.9985650762467175,3.9985650762467193,3.998681814638193,2.0069556978241962,12.309738912151516,8.170357652081286,5.120095321641956,4.347283777504655,3.2018146779054333,1.545436026470796,2.3787163616281197,4.540945782664989,1.6288476774183245,4.080238899283935,4.523882939370636,2.5255374485792306,2.1021396322679866,1.7472781546907712,2.154173053683155,1.4855247382990147,3.727730118646754,2.675245741693549] |F3 |
[3.6440219847273774,1.9997162599711977,6.133140101285886,4.000484218041817,1.9993033836778378,1.9993409073190969,1.9993033836778369,1.9992825381233588,1.9992825381233597,3.998681814638193,4.461775458935025,14.171040042196715,7.00316370178396,5.749303594191232,4.347283777504655,2.198836586031442,4.297066024821238,3.2227770060768073,2.8237814110689845,3.1819349977474243,4.042105825458852,4.806625623081301,3.100992812462546,2.8139453965397445,1.7341749790757055,2.0298607652053304,1.130337012068557,2.982184094917403,0.8917485805645163] |F1 |
[3.6440219847273774,1.9997162599711977,6.378465705337321,2.0002421090209084,3.9986067673556756,1.9993409073190969,1.9993033836778369,1.9992825381233588,3.9985650762467193,1.9993409073190964,2.3124027368021087,11.744161783536812,8.170357652081286,3.840387255317445,2.385235346949457,3.857608045669197,4.523227394548671,3.5680745424421794,1.4882091220498703,2.954653926479751,1.6397221744785908,5.37211099050263,0.5953205733035829,0.5579755900490561,1.752229956547976E-5,1.889933843314045E-5,1.8677575944978003E-5,3.9762454598898707,3.5669943222580653]|F4 |
+----------------------------------------------------------------------------------------------------------

1) Logistic Regression Model

In [44]:
lr = LR(featuresCol='indexedFeatures', labelCol='indexedLabel', maxIter=50, regParam=0.01)

Let's define pipeline architecture

In [46]:
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel",
                               labels=labelIndexer.labels)

pipeline = Pipeline(stages=[labelIndexer, featureIndexer, lr,labelConverter])
model1 = pipeline.fit(train_data) ##train model logistique regression

Prediction with test_data.

In [48]:
predictions = model1.transform(test_data)
predictions.select("features","label","predictedLabel").show(5)

+--------------------+-----+--------------+
 features|label|predictedLabel|
+--------------------+-----+--------------+
[3.64402198472737...| F1| F1|
[3.64402198472737...| F2| F2|
[3.64402198472737...| F3| F3|
[3.64402198472737...| F1| F1|
[3.64402198472737...| F4| F4|
+--------------------+-----+--------------+
only showing top 5 rows

In [49]:
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("accuracy:", accuracy)

accuracy: 0.9892857142857143

We get a good accuracy

2)Decision Tree

In [52]:
dt =  DecisionTreeClassifier(labelCol='indexedLabel', featuresCol='indexedFeatures')


In [53]:
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel",
                               labels=labelIndexer.labels)
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, dt,labelConverter])
model2 = pipeline.fit(train_data)

In [54]:
predictions = model2.transform(test_data)
predictions.select("features","label","predictedLabel").show(5)

+--------------------+-----+--------------+
 features|label|predictedLabel|
+--------------------+-----+--------------+
[3.64402198472737...| F1| F1|
[3.64402198472737...| F2| F2|
[3.64402198472737...| F3| F3|
[3.64402198472737...| F1| F1|
[3.64402198472737...| F4| F4|
+--------------------+-----+--------------+
only showing top 5 rows

In [55]:
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Accuracy:",accuracy)

Accuracy: 1.0